In [2]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [3]:
from gensim import corpora, models
import pandas as pd

# データの読み込み

In [4]:
import matplotlib.pyplot as plt

In [5]:
target_data_food = pd.read_csv("med_cat1_food_data_target_user10_2500.csv", index_col=0)

In [6]:
target_data_food

,receipt_num,receipt_buydate,G_gyoutai_name,cat1,cat4,id
Unnamed: 0,,,,,,
273,57412982.0,2014-08-30,薬局・ドラッグストア,食料品,たばこ,9015.0
337,56277527.0,2014-08-04,薬局・ドラッグストア,食料品,たばこ,32473.0
338,56802603.0,2014-08-12,薬局・ドラッグストア,食料品,たばこ,32473.0
407,56550070.0,2014-08-11,薬局・ドラッグストア,食料品,たばこ,3623.0
415,57237065.0,2014-08-26,薬局・ドラッグストア,食料品,たばこ,5271.0
...,...,...,...,...,...,...
1197997,57003022.0,2014-08-23,薬局・ドラッグストア,食料品,その他酒類,31948.0
1198005,56521414.0,2014-08-08,薬局・ドラッグストア,食料品,菓子パン・調理パン,26351.0
1198062,56778067.0,2014-08-16,薬局・ドラッグストア,食料品,チョコレ－ト,27211.0


# コーパス作成

In [7]:
#全体のコーパス作成
grouped_cat4_data = target_data_food.groupby('id')['cat4'].apply(lambda x: [product.strip() for product in x]).reset_index()
user_cat4 = grouped_cat4_data['cat4'].tolist()

In [8]:
#年月別のコーパス作成
target_data_food['receipt_buydate'] = pd.to_datetime(target_data_food['receipt_buydate'])
grouped_cat4_data_ym = target_data_food.groupby([target_data_food['receipt_buydate'].dt.to_period("M"), 'id'])['cat4'].apply(lambda x: [product.strip() for product in x]).reset_index()
user_cat4_ym = grouped_cat4_data_ym['cat4'].tolist()

In [9]:
grouped_cat4_data_ym

,receipt_buydate,id,cat4
0,2014-04,2.0,"[栄養バランス食品, キャンディ, ワイン, チ－ズ, チ－ズ, チ－ズ, 栄養バランス食品..."
1,2014-04,4.0,"[キャンディ, チョコレ－ト]"
2,2014-04,5.0,"[豆腐類, 炭酸飲料, 炭酸飲料, ヨ－グルト, ヨ－グルト, ドレッシング, ドレッシング..."
3,2014-04,6.0,"[納豆, ヨ－グルト, 納豆, かまぼこ, 乳酸菌飲料, 砂糖, ヨ－グルト, こうや豆腐,..."
4,2014-04,8.0,"[アイスクリ－ム, スナック, 炭酸飲料, ミネラルウォ－タ類, コ－ヒ－ドリンク, 液体茶..."
...,...,...,...
130150,2015-03,40332.0,"[液体茶, 洋風食品, 液体茶, 液体茶, 液体茶, 液体茶, その他菓子, 果汁飲料, 低..."
130151,2015-03,40338.0,"[デザ－ト類, 納豆, スナック, スナック, チョコレ－ト, チョコレ－ト, チョコレ－ト..."
130152,2015-03,40343.0,"[お茶漬けの素, サイダ－, わかめ・こんぶ類, スナック, キャンディ, チョコレ－ト, ..."
130153,2015-03,40349.0,"[ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺, ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺, 低カロリ－甘味料, ミネラルウォ－タ類..."


In [10]:
grouped_cat4_data_ym.groupby(grouped_cat4_data_ym['receipt_buydate']).count()

,id,cat4
receipt_buydate,,
2014-04,10224,10224
2014-05,10826,10826
2014-06,10915,10915
2014-07,11015,11015
2014-08,10848,10848
2014-09,10876,10876
2014-10,10902,10902
2014-11,10954,10954
2014-12,11119,11119


In [11]:
# 月ごとに分割して辞書に保存
#grouped_cat4_data_ym['receipt_buydate'] = pd.to_datetime(grouped_cat4_data_ym['receipt_buydate'])
monthly_data = {month: data for month, data in grouped_cat4_data_ym.groupby(grouped_cat4_data_ym['receipt_buydate'])['cat4']}


In [12]:
for month, data in monthly_data.items():
    print(month)

2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12
2015-01
2015-02
2015-03


In [13]:
monthly_data[pd.Period('2014-04', freq='M')]

0        [栄養バランス食品, キャンディ, ワイン, チ－ズ, チ－ズ, チ－ズ, 栄養バランス食品...
1                                          [キャンディ, チョコレ－ト]
2        [豆腐類, 炭酸飲料, 炭酸飲料, ヨ－グルト, ヨ－グルト, ドレッシング, ドレッシング...
3        [納豆, ヨ－グルト, 納豆, かまぼこ, 乳酸菌飲料, 砂糖, ヨ－グルト, こうや豆腐,...
4        [アイスクリ－ム, スナック, 炭酸飲料, ミネラルウォ－タ類, コ－ヒ－ドリンク, 液体茶...
                               ...                        
10219    [紅茶ドリンク, 煎餅・あられ, 煎餅・あられ, 煎餅・あられ, サイダ－, 洋風食品, 洋...
10220    [デザ－ト類, チョコレ－ト, 納豆, 麦茶, デザ－ト類, キャンディ, 菓子パン・調理パ...
10221                                             [チョコレ－ト]
10222    [チョコレ－ト, ミネラルウォ－タ類, 低カロリ－甘味料, 牛乳, 牛乳, ベ－コン, チョ...
10223    [チョコレ－ト, 液体茶, ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺, 液体茶, ベビ－フ－ド, ベビ－フ－ド...
Name: cat4, Length: 10224, dtype: object

# 辞書の作成

In [14]:
texts = target_data_food['cat4'].tolist()
processed_docs = [[str(doc).lower().strip()] for doc in texts]
dictionary = corpora.Dictionary(processed_docs)

In [15]:
for doc in dictionary.values():
    print(doc)

たばこ
チ－ズ
スナック
その他酒類
その他汎用調味料
その他飲料
ベビ－フ－ド
スポ－ツドリンク
アイスクリ－ム
デザ－ト類
チュ－インガム
コ－ヒ－ドリンク
紅茶ドリンク
マヨネ－ズ
炭酸飲料
ビ－ル
液体茶
米飯類
冷凍調理
菓子パン・調理パン
コ－ラ
ワイン
ココア
焼酎
袋インスタント麺
洋風食品
牛乳
その他スプレッド類
ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺
生麺・ゆで麺
つまみ類
煎餅・あられ
まぜご飯の素
栄養ドリンク
１００％ジュ－ス
畜肉ソ－セ－ジ
チョコレ－ト
つゆ・煮物料理の素
キャンディ
食酢
パスタソ－ス
プレミックス
魚貝類缶詰
乾麺
小麦粉
その他合わせ調味料
佃煮
ゴマ油
ミネラルウォ－タ類
バタ－
ジャム・ママレ－ド
ﾒﾆｭ-専用料理の素
香辛料
ヨ－グルト
漬物
味噌汁・吸物類
冷凍農産
シチュ－ベ－ス
マ－ガリン類
ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-
栄養バランス食品
育児用粉ミルク
食パン
野菜ジユ－ス
ふりかけ
サイダ－
蜂蜜
ス－プ類
豆乳
こうや豆腐
ベ－コン
乳酸飲料
海苔
豆腐類
ﾚｷﾞｭﾗ-ｺ-ﾋ-
乳酸菌飲料
ソ－ス
焼肉ｼｬﾌﾞｼｬﾌﾞのﾀﾚ
みりん風調味料
煮豆
トマトジュ－ス
料理酒
納豆
シリアル類
ウィスキ－
果汁飲料
風味調味料
美容・健康ドリンク
米
キャラメル
その他菓子
ドレッシング
畜肉ハム
魚肉ソ－セ－ジ
野菜缶詰
介護食品
揚げ物
フル－ツ缶詰
醤油
味噌
日本酒
スパゲッティ－
かまぼこ
和風食品
ケチャップ
削り節
ﾌﾚｯｼｭｸﾘ-ﾑ
わかめ・こんぶ類
ｲﾝｽﾀﾝﾄｺ-ﾋ-
畜肉缶詰
ｲﾝｽﾀﾝﾄ ｸﾘ-ﾑ
煮干し
調理用ス－プ
てんぷら粉
カレ－
シロップ類
麦茶
マカロニ類
玩具メ－カ－菓子
日本茶
その他茶
パン粉
その他缶詰
食用塩
サラダ油・天ぷら油
砂糖
その他食用油
紅茶
その他調理食品
春雨・くず切り
低カロリ－甘味料
中国茶
ぽん酢
お茶漬けの素
もずく・めかぶ
ちくわ
パウチ入り食材
焼豚
中華風食品
はんぺん
エッセンス類
シチュ－
本みりん
魚肉ハム
その他麺類
唐揚げ粉
ﾎｲｯﾌﾟｸﾘ-ﾑ
その他食品
麦芽飲料
フリ－マルチ
スキムミルク
その他練り製品
練りミルク
冷凍水産
うまみ調味料
海藻サラダ
その他冷凍食品
液体だし
複合調味料
鍋補完材


# コーパス関数

In [16]:
#コーパスを作成する関数
def create_corpus(m_data):
    corpus = [dictionary.doc2bow(doc) for doc in m_data]
    return corpus


In [17]:
#月別のデータのコーパス作成

#月別のコーパスを保存する辞書
corpus_dict = {}

for month, data in monthly_data.items():
    print(f'{month}のデータ完了')
    print()
    corpus_dict[month] = data


2014-04のデータ完了

2014-05のデータ完了

2014-06のデータ完了

2014-07のデータ完了

2014-08のデータ完了

2014-09のデータ完了

2014-10のデータ完了

2014-11のデータ完了

2014-12のデータ完了

2015-01のデータ完了

2015-02のデータ完了

2015-03のデータ完了



In [18]:
corpus_dict

{Period('2014-04', 'M'): 0        [栄養バランス食品, キャンディ, ワイン, チ－ズ, チ－ズ, チ－ズ, 栄養バランス食品...
 1                                          [キャンディ, チョコレ－ト]
 2        [豆腐類, 炭酸飲料, 炭酸飲料, ヨ－グルト, ヨ－グルト, ドレッシング, ドレッシング...
 3        [納豆, ヨ－グルト, 納豆, かまぼこ, 乳酸菌飲料, 砂糖, ヨ－グルト, こうや豆腐,...
 4        [アイスクリ－ム, スナック, 炭酸飲料, ミネラルウォ－タ類, コ－ヒ－ドリンク, 液体茶...
                                ...                        
 10219    [紅茶ドリンク, 煎餅・あられ, 煎餅・あられ, 煎餅・あられ, サイダ－, 洋風食品, 洋...
 10220    [デザ－ト類, チョコレ－ト, 納豆, 麦茶, デザ－ト類, キャンディ, 菓子パン・調理パ...
 10221                                             [チョコレ－ト]
 10222    [チョコレ－ト, ミネラルウォ－タ類, 低カロリ－甘味料, 牛乳, 牛乳, ベ－コン, チョ...
 10223    [チョコレ－ト, 液体茶, ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺, 液体茶, ベビ－フ－ド, ベビ－フ－ド...
 Name: cat4, Length: 10224, dtype: object,
 Period('2014-05', 'M'): 10224                    [ミネラルウォ－タ類, ス－プ類, ス－プ類, スポ－ツドリンク]
 10225    [栄養バランス食品, キャラメル, キャンディ, キャンディ, キャンディ, チ－ズ, チ－...
 10226    [ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺, 米飯類, 冷凍調理, 冷凍調理, 冷凍調理, チ－ズ, 菓子パン...
 10227    [ヨ－グルト, キャンディ, 豆腐類, 牛乳, 牛乳, 牛乳, 炭酸飲料, 袋インスタント麺...
 10228   

# ldaの実装

In [19]:
from gensim.models import LdaModel

In [20]:
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    print(month)
    for topic_num , term in lda_model.print_topics():
        top_terms = term.split(" + ")
        top_5_terms = " + ".join(top_terms[0:5])
        print(f"({topic_num}, {top_5_terms})")
    print()

2014-04
(0, 0.181*"チョコレ－ト" + 0.174*"アイスクリ－ム" + 0.148*"ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺" + 0.086*"ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-" + 0.078*"スナック")
(1, 0.143*"コ－ヒ－ドリンク" + 0.104*"液体茶" + 0.076*"栄養ドリンク" + 0.060*"炭酸飲料" + 0.047*"美容・健康ドリンク")
(2, 0.133*"煎餅・あられ" + 0.092*"ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-" + 0.082*"食パン" + 0.068*"牛乳" + 0.051*"キャンディ")
(3, 0.123*"菓子パン・調理パン" + 0.100*"豆腐類" + 0.055*"スナック" + 0.053*"生麺・ゆで麺" + 0.049*"食パン")
(4, 0.322*"菓子パン・調理パン" + 0.054*"牛乳" + 0.048*"ビ－ル" + 0.044*"ヨ－グルト" + 0.040*"スナック")
(5, 0.273*"冷凍調理" + 0.042*"ミネラルウォ－タ類" + 0.039*"牛乳" + 0.032*"菓子パン・調理パン" + 0.025*"スナック")
(6, 0.386*"栄養バランス食品" + 0.093*"ス－プ類" + 0.043*"たばこ" + 0.038*"チョコレ－ト" + 0.034*"菓子パン・調理パン")
(7, 0.189*"キャンディ" + 0.143*"スポ－ツドリンク" + 0.101*"ヨ－グルト" + 0.087*"その他酒類" + 0.064*"チョコレ－ト")
(8, 0.394*"スナック" + 0.115*"デザ－ト類" + 0.040*"ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-" + 0.034*"チョコレ－ト" + 0.026*"炭酸飲料")
(9, 0.175*"ベビ－フ－ド" + 0.144*"牛乳" + 0.088*"食パン" + 0.074*"納豆" + 0.069*"ヨ－グルト")

2014-05
(0, 0.156*"牛乳" + 0.112*"食パン" + 0.086*"ヨ－グルト" + 0.068*"豆腐類" + 0.052*"納豆")
(1, 0.430*"スナック" + 0.097*"チョコレ－ト" + 0.082*"煎餅・あら

In [21]:
# # LDAモデルの学習
# # num_topics = 10
# # lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, passes=20, iterations=400)

# # 各ドキュメントのトピック割合を取得
# doc_topics = [lda_model.get_document_topics(doc) for doc in corpus]

# # 各ドキュメントのトピック割合が最も高いトピックをそのドキュメントのトピックとして割り当て
# doc_topic_labels = [max(topics, key=lambda x: x[1])[0] for topics in doc_topics]

# # トピック割り当ての集計
# topic_counts = pd.Series(doc_topic_labels).value_counts()

# # 可視化
# fig, ax = plt.subplots(figsize=(10, 6))
# topic_counts.plot(kind='bar', ax=ax)
# ax.set_xlabel('Topic')
# ax.set_ylabel('Number of Documents')
# ax.set_title(f'Document Topic Distribution (Total Topics: {num_topics})')

# plt.tight_layout()
# plt.show()

In [22]:
lda_model

In [28]:
all_data = []

# 月ごとにLDAモデルからトピックを抽出し、リストに追加
for month, corpus_org in corpus_dict.items():
    corpus = create_corpus(corpus_org)
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
    for topic_num, term in lda_model.print_topics():
        top_terms = term.split(" + ")
        formatted_terms = ["{}({})".format(word.split("*")[1].replace('"', '').strip(), round(float(word.split("*")[0]), 3)) for word in top_terms[:5]]
        all_data.append([month, topic_num] + formatted_terms)


# データフレームの作成
columns = ['月', 'トピック', '1位', '2位', '3位', '4位', '5位']
df = pd.DataFrame(all_data, columns=columns)

# # CSVファイルへの書き出し
csv_file_path = 'lda_topic/Topics_Data_med.csv'
df.to_csv(csv_file_path, index=False)

In [26]:
df

,月,トピック,1位,2位,3位,4位,5位
0,2014-04,0,キャンディ(0.152),菓子パン・調理パン(0.143),液体茶(0.084),コ－ヒ－ドリンク(0.063),コ－ラ(0.049)
1,2014-04,1,煎餅・あられ(0.19),ビ－ル(0.142),ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-(0.069),チョコレ－ト(0.064),つまみ類(0.062)
2,2014-04,2,菓子パン・調理パン(0.369),食パン(0.118),牛乳(0.036),スナック(0.034),デザ－ト類(0.029)
3,2014-04,3,ヨ－グルト(0.074),ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺(0.064),栄養ドリンク(0.06),カレ－(0.054),牛乳(0.038)
4,2014-04,4,冷凍調理(0.212),スナック(0.071),デザ－ト類(0.061),ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺(0.04),ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-(0.032)
...,...,...,...,...,...,...,...
115,2015-03,5,菓子パン・調理パン(0.381),栄養バランス食品(0.111),スポ－ツドリンク(0.067),食パン(0.033),ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺(0.029)
116,2015-03,6,チョコレ－ト(0.261),スナック(0.174),キャンディ(0.095),デザ－ト類(0.078),ﾋﾞｽｹﾂﾄ&ｸﾗﾂｶ-(0.056)
117,2015-03,7,食パン(0.117),菓子パン・調理パン(0.109),液体茶(0.084),牛乳(0.041),豆腐類(0.041)
118,2015-03,8,スナック(0.325),煎餅・あられ(0.171),ｶｯﾌﾟｲﾝｽﾀﾝﾄ麺(0.061),菓子パン・調理パン(0.05),ヨ－グルト(0.037)
